In [1]:
import pandas as pd
from groq import Groq
import json
from prompts import * 

In [2]:
df = pd.read_excel(r"C:\Users\Aselsan Proje\Desktop\Izzet\Codes\NLP_Health-main\Medical\Data\Processed\merged_dataset.xlsx")
columns = ["TRADE_NAME","FIRM_NAME","MANUFACTURER_RECALL_REASON","ACTION","Recall Type"]
df = df[columns]

# Filter the DataFrame to include only rows where "ACTION" is not NaN
filtered_df = df[df["ACTION"].notna()]

# Sample 5 random rows from the filtered DataFrame with the index reset
random_rows = filtered_df.sample(n=5, ignore_index=True)

# Print the "ACTION" column from the random rows
print(random_rows["ACTION"])

0    Action The firm, Siemens, sent "Urgent Medical...
1    Action Smiths Medical issued an URGENT MEDICAL...
2    Action Siemens Medical Solutions USA, Inc.  in...
3    Action An Urgent Field Safety Notice/Product N...
4    Action On 06/12/2023, Beckman Coulter issued a...
Name: ACTION, dtype: object


In [3]:
random_rows['processed_ACTION'] = random_rows['ACTION'].apply(process_action_text)
print(random_rows[['processed_ACTION']])

                                    processed_ACTION
0  • Use the UFSN as an addendum to the system op...
1  • Take the following actions: Locate all affec...
2  • Please contact your local service organizati...
3  • Take actions to address a safety issue expla...
4  • Inspect tubes routed to error regions and co...


In [4]:
random_rows['processed_RECALL'] = random_rows['MANUFACTURER_RECALL_REASON'].apply(process_recall_text)
print(random_rows[['processed_RECALL']])


                                    processed_RECALL
0  • Improperly loaded sample racks may cause mis...
1  • Delivery disruptions during motor non-runnin...
2  • Workflow interruptions may occur, resulting ...
3  • Delayed display of potential patient movemen...
4  • Incompatible tube types used for patient sam...


In [5]:
random_rows["prompt_input"] = random_rows.apply(create_prompt_input, axis=1)
print(random_rows[['prompt_input']])


                                        prompt_input
0  {"TRADE_NAME": "ADVIA Centaur XP Immunoassay S...
1  {"TRADE_NAME": "smiths medical Medfusion", "FI...
2  {"TRADE_NAME": "Siemens", "FIRM_NAME": "Siemen...
3  {"TRADE_NAME": "ExacTrac Dynamic Software", "F...
4  {"TRADE_NAME": "DxA 5000 (DxA Automation Syste...


In [6]:
random_rows = random_rows.drop(["TRADE_NAME",	"FIRM_NAME",	"MANUFACTURER_RECALL_REASON",	"ACTION",	"Recall Type","processed_RECALL"],axis=1)

In [7]:
random_rows["prompt_input"][0]

'{"TRADE_NAME": "ADVIA Centaur XP Immunoassay System", "FIRM_NAME": "Siemens Healthcare Diagnostics", "processed_RECALL": "\\u2022 Improperly loaded sample racks may cause misreads of Sample IDs (SID) due to actions deviating from the loading instructions.\\n\\u2022 Pushing sample racks too far on the sample entry queue during normal operation may lead to misreads of Sample IDs (SID) due to incorrect actions.\\n\\u2022 Failure to follow exact sample rack loading instructions may result in misreads of Sample IDs (SID) due to inappropriate actions.", "Recall Type": "SoftwareDesignManufacturingProcess"}'

In [11]:
print(random_rows["processed_ACTION"][0])

• Use the UFSN as an addendum to the system operating instructions until further notice and take actions accordingly.
• Review this letter with your Medical Director and take actions to rectify the issue.
• Complete and return the attached FIELD CORRECTION EFFECTIVENESS CHECK within the specified timeframe and take actions to comply with the instructions.
• Retain the letter with your laboratory records and forward the letter to those who may have received this product, taking necessary actions to ensure compliance.
• If you have any questions, contact your Siemens Customer Care Center, your local Siemens technical support representative and/or call the specified number to clarify any actions to be taken.


In [12]:
random_rows.to_excel(r"final.xlsx",index= False)